In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xarray as xr
import glob
import json
from modules.full_pipeline import WeatherBenchDatasetXarrayHealpixTempMultiple, load_data_split
import os
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import torch

In [3]:
def interpolate_coord(ds, lon, lat, npixels):
    assert npixels == len(lon) ==len(lat)
    ds_interp = ds.interp(lon=('node', lon), lat=('node', lat), method='nearest').interpolate_na(dim='node', method='nearest')
    ds_interp = ds_interp.assign_coords(node=np.arange(npixels))
    return ds_interp

# Interpolation

In [11]:
z500 = xr.open_mfdataset('/home/wefeng/equiangular/5.625deg/geopotential_500/raw/*.nc', combine='by_coords')
t850 = xr.open_mfdataset('/home/wefeng/equiangular/5.625deg/temperature_850/raw/*.nc', combine='by_coords')
solar = xr.open_mfdataset('/home/wefeng/equiangular/5.625deg/toa_incident_solar_radiation/raw/*.nc', combine='by_coords')

In [12]:
new_lat = 32
new_lon = 64

npixels = new_lat * new_lon
print(npixels)

2048


In [13]:
z500.lon.min().values

array(0.)

In [14]:
z500.lon.max().values

array(354.375)

In [15]:
new_lon_idx = np.linspace(z500.lon.min(), z500.lon.max(), num=new_lat, endpoint=True)
new_lat_idx = np.linspace(z500.lat.min(), z500.lat.max(), num=new_lon, endpoint=True)

space_lon = np.mean(new_lon_idx[1:] -  new_lon_idx[:-1])
space_lat = np.mean(new_lat_idx[1:] - new_lat_idx[:-1])

new_lat_idx, new_lon_idx = np.meshgrid(new_lat_idx, new_lon_idx, indexing='ij')

new_lon_idx = new_lon_idx.reshape(-1)
new_lat_idx = new_lat_idx.reshape(-1)

lon_idx = xr.DataArray(new_lon_idx, dims=["lon"])
lat_idx = xr.DataArray(new_lat_idx, dims=["lat"])

In [33]:
new_lat_idx

array([-87.1875, -87.1875, -87.1875, ...,  87.1875,  87.1875,  87.1875])

In [34]:
z500_interp = interpolate_coord(z500, lon_idx, lat_idx, npixels)
t850_interp = interpolate_coord(t850, lon_idx, lat_idx, npixels)
solar_interp = interpolate_coord(solar, lon_idx, lat_idx, npixels)

In [35]:
z500_interp

<xarray.Dataset>
Dimensions:  (node: 2048, time: 350640)
Coordinates:
    level    int32 500
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
    lat      (node) float64 -87.19 -87.19 -87.19 -87.19 ... 87.19 87.19 87.19
    lon      (node) float64 0.0 11.43 22.86 34.29 ... 320.1 331.5 342.9 354.4
  * node     (node) int64 0 1 2 3 4 5 6 7 ... 2041 2042 2043 2044 2045 2046 2047
Data variables:
    z        (time, node) float64 dask.array<chunksize=(8760, 2048), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-10 20:33:23 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

In [128]:
# z500_interp.to_netcdf('/home/wefeng/equiangular/5.625deg/geopotential_500/geopotential_500_5.625deg.nc')
# t850_interp.to_netcdf('/home/wefeng/equiangular/5.625deg/temperature_850/temperature_850_5.625deg.nc')
# solar_interp.to_netcdf('/home/wefeng/equiangular/5.625deg/toa_incident_solar_radiation/toa_incident_solar_radiation_5.625deg.nc')

# Obs

In [4]:
from modules.test import create_iterative_observations_eq

In [5]:
datadir = "/nfs_home/wefeng/equiangular/"
input_dir = datadir + "5.625deg/"

chunk_size = 521

train_years = ('1990', '2012')#('1979', '2012')
val_years = ('2013', '2016')
test_years = ('2017', '2018')

nodes = 32 * 64
max_lead_time = 5*24
nb_timesteps = 2

In [6]:
obs = create_iterative_observations_eq(input_dir, test_years, 6, max_lead_time, nb_timesteps)

In [16]:
obs_interp = interpolate_coord(obs, lon_idx, lat_idx, nodes)

In [17]:
obs_interp.to_netcdf('/mnt/scratch/students/wefeng/data/equiangular/5.625deg/observations.nc')

# Constants

In [4]:
constants_non = xr.open_dataset('/home/wefeng/equiangular/5.625deg/constants/constants_5.625deg.nc')

In [5]:
constants_non

<xarray.Dataset>
Dimensions:    (lat: 32, lon: 64)
Coordinates:
  * lat        (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon        (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Data variables:
    orography  (lat, lon) float32 ...
    lsm        (lat, lon) float32 ...
    slt        (lat, lon) float32 ...
    lat2d      (lat, lon) float64 ...
    lon2d      (lat, lon) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-11-21 15:25:20 GMT by grib_to_netcdf-2.14.0: /opt/ecmw...

In [6]:
constants_standard = (constants_non - constants_non.mean()) / constants_non.std()

In [7]:
constants_standard = constants_standard.rename({'orography': 'orog'})

In [8]:
new_lat = 32
new_lon = 64

npixels = new_lat * new_lon
print(npixels)

2048


In [10]:
new_lon_idx = np.linspace(constants_non.lon.min(), constants_non.lon.max(), num=new_lat, endpoint=True)
new_lat_idx = np.linspace(constants_non.lat.min(), constants_non.lat.max(), num=new_lon, endpoint=True)

space_lon = np.mean(new_lon_idx[1:] -  new_lon_idx[:-1])
space_lat = np.mean(new_lat_idx[1:] - new_lat_idx[:-1])

new_lat_idx, new_lon_idx = np.meshgrid(new_lat_idx, new_lon_idx, indexing='ij')

new_lon_idx = new_lon_idx.reshape(-1)
new_lat_idx = new_lat_idx.reshape(-1)

lon_idx = xr.DataArray(new_lon_idx, dims=["lon"])
lat_idx = xr.DataArray(new_lat_idx, dims=["lat"])

In [11]:
constants_standard = interpolate_coord(constants_standard, lon_idx, lat_idx, npixels)

In [12]:
constants_standard.to_netcdf('/home/wefeng/equiangular/5.625deg/constants/constants_5.625deg_standardized.nc')

In [14]:
orog = constants_standard['orog']
lsm = constants_standard['lsm']
slt = constants_standard['slt']
lats = constants_standard['lat2d']

In [17]:
constants_tensor = torch.tensor(xr.merge([orog, lats, lsm, slt], compat='override').to_array().values, dtype=torch.float)

In [23]:
constants_tensor.shape

torch.Size([4, 2048])

In [22]:
torch.mean(constants_tensor, dim=1, keepdim=True)

torch.Size([4, 1])

# Graph

In [24]:
import pygsp
from scipy import sparse
import torch

In [25]:
def prepare_laplacian(laplacian):
    """Prepare a graph Laplacian to be fed to a graph convolutional layer
    """

    def estimate_lmax(laplacian, tol=5e-3):
        r"""Estimate the largest eigenvalue of an operator."""
        lmax = sparse.linalg.eigsh(laplacian, k=1, tol=tol,
                                   ncv=min(laplacian.shape[0], 10),
                                   return_eigenvectors=False)
        lmax = lmax[0]
        lmax *= 1 + 2 * tol  # Be robust to errors.
        return lmax

    def scale_operator(L, lmax, scale=1):
        r"""Scale the eigenvalues from [0, lmax] to [-scale, scale]."""
        I = sparse.identity(L.shape[0], format=L.format, dtype=L.dtype)
        L *= 2 * scale / lmax
        L -= I
        return L

    lmax = estimate_lmax(laplacian)
    laplacian = scale_operator(laplacian, lmax)

    laplacian = sparse.coo_matrix(laplacian)

    # PyTorch wants a LongTensor (int64) as indices (it'll otherwise convert).
    indices = np.empty((2, laplacian.nnz), dtype=np.int64)
    np.stack((laplacian.row, laplacian.col), axis=0, out=indices)
    indices = torch.from_numpy(indices)

    laplacian = torch.sparse_coo_tensor(indices, laplacian.data, laplacian.shape)
    laplacian = laplacian.coalesce()  # More efficient subsequent operations.
    return laplacian

In [39]:
G = pygsp.graphs.SphereEquiangular(bandwidth=(8, 16))
G.compute_laplacian("normalized")
laplacian = prepare_laplacian(G.L.astype(np.float32))

In [40]:
laplacian.shape

torch.Size([512, 512])

In [71]:
G = pygsp.graphs.SphereEquiangular(bandwidth=(1, 2))
G.compute_laplacian("normalized")
laplacian = prepare_laplacian(G.L.astype(np.float32))

In [1]:
from deepsphere.utils.samplings import equiangular_dimension_unpack
from pygsp.graphs import SphereEquiangular

In [2]:
equiangular_dimension_unpack(2048, 2)

(32, 64)

In [4]:
SphereEquiangular(bandwidth=(16, 32))

SphereEquiangular(n_vertices=2048, n_edges=4096)